Basic code for antibody-based simulations
===========

## basic imports ##

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from mdtraj.reporters import *
from functional import seq

from pathlib import Path
import sys
from sys import stdout
import inspect

In [2]:
debug_local = True#False
local = Path("..").resolve()
code = local / "mm"
data  = local / "data"
input = data / "input"
output = data / "output"

if debug_local and code.exists():
    sys.path.insert(0, code.as_posix())
    print("extending pathes with local yspecies")
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

extending pathes with local yspecies
['/data/sources/antibody-mm/mm', '/data/sources/antibody-mm/notebooks', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python39.zip', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/lib-dynload', '', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
def children(p: Path):
    """
    files and subfolders in the folder as sequence
    :param p:
    :return:
    """
    return seq(list(p.iterdir()))
children(input).for_each(print)

/data/sources/antibody-mm/data/input/1adq_short.pdb
/data/sources/antibody-mm/data/input/1adq_fixed.pdb


In [4]:
children(input).for_each(print)

/data/sources/antibody-mm/data/input/1adq_short.pdb
/data/sources/antibody-mm/data/input/1adq_fixed.pdb


## Setting simulation up ##

### Loading the file ###

In [5]:
#pdb_name = "1adq_short.pdb"
#pdb_name = "1adq_short.pdb"
pdb_name = "1adq_fixed.pdb"
pdb_path = (input / pdb_name) 
pdb_stem =pdb_path.stem
pdb = PDBFile(pdb_path.as_posix())
modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater() #for implicit solvent


#### Setting up the force field and system ####

In [6]:
#forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
#system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)

forcefield = ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3pfb.xml')
modeller.addHydrogens(forcefield)
""

''

In [7]:
system = forcefield.createSystem(modeller.topology, 
                                 nonbondedMethod=CutoffNonPeriodic,
        nonbondedCutoff=1.2*nanometer,
        switchDistance=1.0*nanometer, 
        constraints=HBonds, 
        implicitSolvent=app.GBn2,
        implicitSolventSaltConc=0.15*moles/liter)

In [8]:
temperature = 310 * kelvin
integrator = LangevinMiddleIntegrator(temperature, 1 / picosecond, 0.002*picoseconds)

#### Setting up simulation ####

In [9]:
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()

#### Adding reporters ####

In [10]:
report_interval = 1000
checkpoint_interval = report_interval * 10
report_pdb: bool = True #pdb reporter can be heavy


In [11]:
simulation.reporters.append(PDBReporter((output / f'{pdb_stem}.dcd').as_posix(), report_interval))
simulation.reporters.append(DCDReporter((output / f'{pdb_stem}.dcd').as_posix(), report_interval))
simulation.reporters.append(StateDataReporter(stdout, checkpoint_interval, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, step=True,temperature=True, separator='\t'))
simulation.reporters.append(StateDataReporter((output / f'{pdb_stem}_log.tsv').as_posix(), report_interval, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, step=True,temperature=True, separator='\t'))
simulation.reporters.append(HDF5Reporter((output / f'{pdb_stem}_log.h5').as_posix(), report_interval, coordinates=True, time=True, cell=True, potentialEnergy=True, kineticEnergy=True, temperature=True))
simulation.reporters.append(CheckpointReporter((output / f'{pdb_stem}_results.chk').as_posix(), checkpoint_interval))


## Running the simulation ##

In [12]:
steps = 50000 #50000000 # to get 10 nanoseconds

In [13]:
simulation.step(steps)

/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


#"Step"	"Potential Energy (kJ/mole)"	"Kinetic Energy (kJ/mole)"	"Total Energy (kJ/mole)"	"Temperature (K)"
10000	38089.3359375	31800.37828320451	69889.71422070451	310.97702149528965
20000	36957.98828125	31811.312096182723	68769.30037743272	311.08394363826767
30000	37177.3359375	31931.08851265651	69108.42445015651	312.25524144198187
40000	36654.65625	31821.49870712147	68476.15495712147	311.18355886613233
50000	36728.69921875	32122.702610051725	68851.40182880173	314.12904246264276


In [14]:
finalpositions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, finalpositions, open((output / f'{pdb_stem}.pdb').as_posix(), 'w'))

simulation.saveState('finalstate.xml')
simulation.saveCheckpoint('finalstate.chk')

simulation.saveState((output / f'{pdb_stem}_results.xml').as_posix())
simulation.saveCheckpoint((output / f'{pdb_stem}_results.chk').as_posix())